# CineSimile: DM Project - Movie Recommendation System

In [1]:
import json
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid")

pd.set_option('display.max_rows', 100)

In [10]:
import pickle
import glob

base_path = os.path.dirname(os.getcwd())
master_data_path = os.path.abspath(os.path.join(base_path,'master-data'))
data_path = os.path.abspath(os.path.join(base_path,'data'))
images_path = os.path.abspath(os.path.join(base_path,'dm-final-report', 'images'))   
# tsv_files = glob.glob(os.path.join(master_data_path,"*.tsv.gz"))

# for file in tsv_files:
#     print(file)
#     pickle.dump(pd.read_table(file,sep="\t",low_memory=False, na_values=["\\N","nan"]),
#                 open(file[:-7]+".sav","wb"))

## Dataset overview

Titles Dataset

In [ ]:
# read pickle pandas
basics_df = pd.read_pickle(os.path.join(master_data_path,"title.basics.sav"))
basics_df

In [ ]:
# check missing values and rename columns
print(basics_df.isnull().sum().reset_index().rename(columns={'index':'column', 0:"missing_values"}).to_latex(index=False))

In [ ]:
import plotly.express as px


fig = px.pie(basics_df, names='titleType', hole=0.3,
             title='Title Types Distribution',
             width=800, height=800)


fig.update_traces(textposition='outside', textinfo='percent+label',
                  marker=dict(line=dict(color='#000000', width=2)),
                  pull=0.05, 
                  rotation=140) 


fig.update_layout(legend_title='Title Types',
                  legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5),
                  margin=dict(t=0, l=0, r=0, b=0))


fig.show()

In [ ]:
# cleaning the basics dataset 
# remove types which are not movies
basics_movies_df = basics_df[basics_df['titleType'].isin(['movie'])].copy()
# drop endYear column
basics_movies_df.drop('endYear',axis=1,inplace=True)
basics_movies_df.describe(include='all')

In [ ]:
basics_movies_df.isnull().sum().reset_index().rename(columns={'index':'column', 0:"missing_values"})

In [ ]:
# drop rows with missing values
basics_movies_df.dropna(inplace=True)

In [ ]:
# set datatypes for the columns
# set startYear to datetime year
basics_movies_df['startYear'] = pd.to_datetime(basics_movies_df['startYear'],format='%Y',errors='coerce').dt.year
# set isAdult to boolean
basics_movies_df['isAdult'] = basics_movies_df['isAdult'].astype(bool)
# set runtimeMinutes to integer
basics_movies_df['runtimeMinutes'] = pd.to_numeric(basics_movies_df['runtimeMinutes'],errors='coerce')
basics_movies_df.describe(include='all')

In [ ]:
basics_movies_df.loc[basics_movies_df['runtimeMinutes']==0,'runtimeMinutes'] = 1

In [ ]:
# check for distribution of runtimeMinutes for each type
print(basics_movies_df[basics_movies_df["titleType"] == "movie"]["runtimeMinutes"].describe(include='all'))
print()

In [ ]:
import plotly.express as px


fig = px.pie(basics_movies_df, names='titleType', hole=0.3,
             title='Title Types Distribution (Movies Only)',
             width=1000, height=1000)

# Customizing the pie chart to closely match the Matplotlib customization
fig.update_traces(textposition='outside', textinfo='percent+label',
                  marker=dict(line=dict(color='#000000', width=2)),
                  pull=0.05, # Slightly separate the slices
                  rotation=120) # Rotate the pie chart

# Updating layout for a cleaner look, positioning the legend at the bottom
fig.update_layout(legend_title='Title Types',
                  legend=dict(orientation="h", yanchor="top", y=-0.2, xanchor="center", x=0.5),
                  margin=dict(t=0, l=0, r=0, b=0), font=dict(size=14))


fig.show()

In [ ]:
import plotly.express as px


fig = px.box(basics_movies_df, x='titleType', y='runtimeMinutes',
             category_orders={"titleType": ["short", "movie", "tvMovie"]},
             title='Distribution of Movie Runtime Minutes by Title Type',
             log_y=True)  # Setting the y-axis to logarithmic scale

fig.update_layout(
    xaxis_title='Title Type',
    yaxis_title='Runtime Minutes (Log Scale)',
    title_font_size=20,  # Increase title font size
    font=dict(size=14),  # Increase general font size for axis titles, tick labels, etc.
    width=800,  # Adjust figure width
    height=600,  # Adjust figure height
    margin=dict(l=40, r=40, t=60, b=40)  # Adjust margins to make the plot more compact
)

fig.show()


In [ ]:
# set datatypes for the columns
# set startYear to datetime year
basics_movies_df['startYear'] = pd.to_datetime(basics_movies_df['startYear'],format='%Y',errors='coerce').dt.year
# set isAdult to boolean
basics_movies_df['isAdult'] = basics_movies_df['isAdult'].astype(bool)
# set runtimeMinutes to integer
basics_movies_df['runtimeMinutes'] = pd.to_numeric(basics_movies_df['runtimeMinutes'],errors='coerce')

# write this to a pickle file
basics_movies_df.to_pickle(os.path.join(data_path,"title.basics.cleaned.sav"))

Read Title Basics Cleaned Dataset (Pickled)

In [3]:
# read pickle pandas
basics_movies_df = pd.read_pickle(os.path.join(data_path,"title.basics.cleaned.sav"))
basics_movies_df.describe(include='all')

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
count,421608,421608,421608,421608,421608,421608.000000,421608.000000,421608
unique,421608,1,349626,359826,2,NaN,NaN,1380
top,tt0000009,movie,Home,Home,False,NaN,NaN,Drama
freq,1,421608,41,35,415059,NaN,NaN,75638
mean,NaN,NaN,NaN,NaN,NaN,1997.132877,90.326094,NaN
std,NaN,NaN,NaN,NaN,NaN,26.394415,146.435676,NaN
min,NaN,NaN,NaN,NaN,NaN,1894.000000,1.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,1983.000000,75.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,2009.000000,90.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,2016.000000,100.000000,NaN


In [ ]:
basics_movies_df.columns

In [ ]:
basics_movies_df
# get the adult movies for short movies
basics_movies_df[(basics_movies_df["titleType"] == "short") & (basics_movies_df["isAdult"] == True)]

In [ ]:
import plotly.express as px

# Calculate the percentage of each titleType with respect to isAdult (True or False)
title_type_counts = basics_movies_df.groupby(['isAdult', 'titleType']).size().unstack(fill_value=0)
title_type_percents = title_type_counts.divide(title_type_counts.sum(axis=0), axis=1) * 100

# Reset index to use isAdult in the plot
title_type_percents = title_type_percents.reset_index()

# Melt the dataframe for easier plotting with plotly
melted_df = title_type_percents.melt(id_vars='isAdult', var_name='Title Type', value_name='Percentage')

# Plot
fig = px.bar(melted_df, x='Title Type', y='Percentage', color='isAdult', barmode='stack',
             labels={'isAdult': 'Is Adult'}, title='Percentage of Each Title Type by Adult Content', log_y=True)

# Update layout for a cleaner look
fig.update_layout(xaxis_title='Title Type', yaxis_title='Percentage (Log Scale)',
                  title_font_size=20, font=dict(size=14), width=800, height=600,
                  margin=dict(l=40, r=40, t=60, b=40))

fig.show()

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np


temp = basics_movies_df['genres'].dropna()
vec = CountVectorizer(token_pattern='(?u)\\b[\\w-]+\\b', analyzer='word').fit(temp)
bag_of_genres = vec.transform(temp)
unique_genres = vec.get_feature_names_out()
np.array(unique_genres)

array(['action', 'adult', 'adventure', 'animation', 'biography', 'comedy',
       'crime', 'documentary', 'drama', 'family', 'fantasy', 'film-noir',
       'game-show', 'history', 'horror', 'music', 'musical', 'mystery',
       'news', 'reality-tv', 'romance', 'sci-fi', 'sport', 'talk-show',
       'thriller', 'war', 'western'], dtype=object)

In [11]:
import plotly.express as px
import pandas as pd


genres = pd.DataFrame(bag_of_genres.todense(), columns=unique_genres, index=temp.index)
sorted_genres_perc = 100 * pd.Series(genres.sum()).sort_values(ascending=False) / genres.shape[0]

# Convert Series to DataFrame for Plotly Express
df_sorted_genres_perc = sorted_genres_perc.reset_index()
df_sorted_genres_perc.columns = ['Genre', 'Percentage of Films']

# Plot using Plotly Express
fig = px.bar(df_sorted_genres_perc, y='Genre', x='Percentage of Films', orientation='h',
             labels={'Percentage of Films': 'Percentage of Films (%) (Log Scale)'},
             height=800, title='Percentage of Films by Genre', log_x=True)

# Update layout for a cleaner look with a white background and ensure text is black
fig.update_layout(
    xaxis_title='Percentage of Films (%) (Log Scale)',
    yaxis_title='Genre',
    title_font_size=20,
    font=dict(size=14, color='black'),
    width=800,
    margin=dict(l=40, r=40, t=60, b=40),
    plot_bgcolor='white',  # Set the plot background to white
    paper_bgcolor='white'  # Set the overall background to white
)


fig.write_image(os.path.join(images_path, 'genres.png'))

In [12]:
basics_movies_df

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,False,1894,45,Romance
146,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,False,1897,100,"Documentary,News,Sport"
573,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,False,1906,70,"Action,Adventure,Biography"
590,tt0000591,movie,The Prodigal Son,L'enfant prodigue,False,1907,90,Drama
677,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,False,1908,120,"Adventure,Fantasy"
...,...,...,...,...,...,...,...,...
10821998,tt9916538,movie,Kuambil Lagi Hatiku,Kuambil Lagi Hatiku,False,2019,123,Drama
10822039,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,False,2015,57,Documentary
10822066,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,False,2007,100,Documentary
10822088,tt9916730,movie,6 Gunn,6 Gunn,False,2017,116,Drama



# Title Ratings Dataset

In [28]:
ratings_df = pd.read_pickle(os.path.join(master_data_path,"title.ratings.sav"))
ratings_df

,tconst,averageRating,numVotes
0,tt0000001,5.7,2037
1,tt0000002,5.7,272
2,tt0000003,6.5,1988
3,tt0000004,5.4,178
4,tt0000005,6.2,2746
...,...,...,...
1417751,tt9916730,7.0,12
1417752,tt9916766,7.1,23
1417753,tt9916778,7.2,36
1417754,tt9916840,7.0,9


In [29]:
ratings_df.describe(include='all')

,tconst,averageRating,numVotes
count,1417756,1.417756e+06,1.417756e+06
unique,1417756,NaN,NaN
top,tt0000001,NaN,NaN
freq,1,NaN,NaN
mean,NaN,6.955015e+00,1.035535e+03
std,NaN,1.385880e+00,1.767701e+04
min,NaN,1.000000e+00,5.000000e+00
25%,NaN,6.200000e+00,1.100000e+01
50%,NaN,7.100000e+00,2.600000e+01
75%,NaN,7.900000e+00,1.010000e+02


In [30]:
# check for missing values in ratings_df
ratings_df.isnull().sum().reset_index().rename(columns={'index':'column', 0:"missing_values"})

,column,missing_values
0,tconst,0
1,averageRating,0
2,numVotes,0


In [ ]:
# merge basics_movies_df and ratings_df based on tconst
movies_ratings_df = pd.merge(basics_movies_df,ratings_df,on='tconst',how='inner')
movies_ratings_df.describe(include='all')

In [ ]:
import plotly.graph_objects as go

counts_yearly = movies_ratings_df.groupby("startYear").agg({"averageRating":[np.median],
                                                     "numVotes":[np.sum,np.size,lambda x: np.sum(x)/np.size(x)]})

max_year = counts_yearly[("numVotes","size")].idxmax().astype(int)

# Data for the first plot
df_size = counts_yearly[("numVotes", "size")]

# Create the first plot
fig1 = go.Figure()

# Add trace
fig1.add_trace(go.Scatter(x=df_size.index, y=df_size, mode='lines', name='Films per Year'))

# Update layout
fig1.update_layout(title_text="Total Number Films per Year", title_x=0.5, height=1000, width=1500, xaxis=dict(
        title="Year",
        tickmode='linear',  # This ensures a linear scale
        dtick=5, 
        tickformat="%Y",  # This ensures that the tick labels are formatted as years
    ), font=dict(size=14))


fig1.show()


In [ ]:
# Data for the second plot
df_sum = counts_yearly[("numVotes", "sum")]
max_count_year = counts_yearly[("numVotes","sum")].idxmax().astype(int)

# Create the second plot
fig2 = go.Figure()

# Add trace
fig2.add_trace(go.Scatter(x=df_sum.index, y=df_sum, mode='lines', name='Voters per Year'))

# Update layout
fig2.update_layout(
    title_text="Total Number of Voters per Year",
    title_x=0.5,
    height=1000,
    width=1500,
    xaxis=dict(
        title="Year",
        tickmode='linear',  # This ensures a linear scale
        dtick=5,  
        tickformat="%Y",  # This ensures that the tick labels are formatted as years
    ), font=dict(size=14)
)


fig2.show()


In [ ]:
# get the average rating box plot distribution
fig = px.box(movies_ratings_df, x='titleType', y='averageRating',
             category_orders={"titleType": ["short", "movie", "tvMovie"]},
             title='Distribution of Average Ratings by Title Type',
             log_y=False)  # Setting the y-axis to logarithmic scale

fig.update_layout( xaxis_title='Title Type', yaxis_title='Average Rating',
                    title_font_size=20, font=dict(size=14), width=800, height=600,
                    margin=dict(l=40, r=40, t=60, b=40))

fig.show()

## Diagnostic Analysis

In [ ]:
movies_ratings_df

In [ ]:
import plotly.figure_factory as ff

features = ['averageRating', 'runtimeMinutes', 'numVotes']  # Add more numeric features as needed
correlation_matrix = movies_ratings_df[features].corr()

# Define the color scale
colors = 'Viridis' 

# Create the heatmap with additional features like color scale
fig = ff.create_annotated_heatmap(
    z=correlation_matrix.to_numpy(),
    x=correlation_matrix.columns.tolist(),
    y=correlation_matrix.index.tolist(),
    annotation_text=correlation_matrix.round(2).astype(str).to_numpy(),
    showscale=True,
    colorscale=colors
)

# Update the layout
fig.update_layout(
    title_text='Correlation Matrix',
    title_x=0.5,
    xaxis_title='Features',
    yaxis_title='Features',
    xaxis=dict(tickmode='array', tickvals=list(range(len(features))), ticktext=features),
    yaxis=dict(tickmode='array', tickvals=list(range(len(features))), ticktext=features)
)

# clean up the layout
fig.update_layout(
    title_font_size=20,
    font=dict(size=14),
    width=800,
    height=800,
    margin=dict(l=40, r=40, t=200, b=40)
)


fig.show()

In [ ]:
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd


genres_df = movies_ratings_df['genres'].str.get_dummies(sep=',')

# Join the genres dataframe with the average ratings
genre_ratings_df = genres_df.join(movies_ratings_df['averageRating'])

# ANOVA for each genre
anova_results = {}
for genre in genres_df.columns:
    grouped_data = genre_ratings_df.groupby(genre)['averageRating']
    # Perform ANOVA and store the results with the genre name
    anova_results[genre] = stats.f_oneway(grouped_data.get_group(1), grouped_data.get_group(0))

# Convert the results to a DataFrame
anova_df = pd.DataFrame(anova_results, index=['F-value', 'p-value']).T

# Display the results
print(anova_df)


tukey_results = pairwise_tukeyhsd(endog=genre_ratings_df['averageRating'], groups=genre_ratings_df['Drama'])
print(tukey_results)


In [ ]:
# Expand the 'genres' column into multiple rows
expanded_genres = movies_ratings_df.drop('genres', axis=1).join(
    movies_ratings_df['genres'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('genre')
)

# Count the number of movies per genre per year
genre_counts_by_year = expanded_genres.groupby(['startYear', 'genre']).size().reset_index(name='counts')

# Bin the years into decades
genre_counts_by_year['decade'] = (genre_counts_by_year['startYear'] // 10) * 10

# Sum genre counts by decade
genre_counts_by_decade = genre_counts_by_year.groupby(['decade', 'genre']).agg({'counts': 'sum'}).reset_index()

# If by decade
fig = px.line(genre_counts_by_decade, x='decade', y='counts', color='genre', title='Genre Popularity Over Decades')

# If by year, replace 'decade' with 'startYear' in the above line

fig.update_layout(xaxis_title='Time', yaxis_title='Number of Movies', legend_title='Genre', font=dict(size=18), width=1000, height=1000,)
fig.show()

In [ ]:
import pandas as pd
import plotly.express as px


movies_expanded = movies_ratings_df.drop('genres', axis=1).join(
    movies_ratings_df['genres'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('genre')
)

# Now, we have one genre per row, we can easily group by genre and calculate average ratings
genre_ratings = movies_expanded.groupby('genre')['averageRating'].describe()[['mean', 'count', '50%', '25%', '75%']]

# Using Plotly to create a box plot
fig = px.box(movies_expanded, x='genre', y='averageRating', title="Genre-wise Distribution of Ratings")


fig.show()


# Title crew dataset

In [13]:
title_crew_df = pd.read_pickle(os.path.join(master_data_path,"title.crew.sav"))
title_crew_df

,tconst,directors,writers
0,tt0000001,nm0005690,NaN
1,tt0000002,nm0721526,NaN
2,tt0000003,nm0721526,NaN
3,tt0000004,nm0721526,NaN
4,tt0000005,nm0005690,NaN
...,...,...,...
10000207,tt9916848,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
10000208,tt9916850,nm1485677,"nm9187127,nm1485677,nm9826385,nm1628284"
10000209,tt9916852,nm1485677,"nm9187127,nm1485677,nm9826385,nm9299459,nm1628284"
10000210,tt9916856,nm10538645,nm6951431


In [16]:
title_crew_df.describe(include='all')

,tconst,directors,writers
count,10000212,6255573,5643919
unique,10000212,892384,1329910
top,tt0000001,nm1203430,nm6352729
freq,1,11416,14800


In [15]:
title_crew_df.isnull().sum().reset_index().rename(columns={'index':'column', 0:"missing_values"})

,column,missing_values
0,tconst,0
1,directors,3744639
2,writers,4356293


In [18]:
# drop rows with missing values

title_crew_df.dropna(inplace=True)

title_crew_df.to_pickle(os.path.join(data_path,"title.crew.cleaned.sav"))

In [19]:
# read pickle pandas
title_crew_df = pd.read_pickle(os.path.join(data_path,"title.crew.cleaned.sav"))
title_crew_df.describe(include='all')

,tconst,directors,writers
count,4832805,4832805,4832805
unique,4832805,709957,1255461
top,tt0000009,nm1203430,nm6352729
freq,1,11284,14363


# Title principals dataset

In [25]:
title_principals_df = pd.read_pickle(os.path.join(master_data_path,"title.principals.sav"))
title_principals_df

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,NaN,"[""Self""]"
1,tt0000001,2,nm0005690,director,NaN,NaN
2,tt0000001,3,nm0005690,producer,producer,NaN
3,tt0000001,4,nm0374658,cinematographer,director of photography,NaN
4,tt0000002,1,nm0721526,director,NaN,NaN
...,...,...,...,...,...,...
84695456,tt9916880,18,nm0996406,director,principal director,NaN
84695457,tt9916880,19,nm1482639,writer,NaN,NaN
84695458,tt9916880,20,nm2586970,writer,books,NaN
84695459,tt9916880,21,nm1594058,producer,producer,NaN


In [21]:
title_principals_df.describe(include='all')

,tconst,ordering,nconst,category,job,characters
count,84695461,8.469546e+07,84695461,84695461,15900285,40920240
unique,9627377,NaN,6222006,13,42401,4029232
top,tt0398022,NaN,nm0438471,actor,producer,"[""Self""]"
freq,75,NaN,32755,20297734,6290523,5157589
mean,NaN,7.000367e+00,NaN,NaN,NaN,NaN
std,NaN,5.142924e+00,NaN,NaN,NaN,NaN
min,NaN,1.000000e+00,NaN,NaN,NaN,NaN
25%,NaN,3.000000e+00,NaN,NaN,NaN,NaN
50%,NaN,6.000000e+00,NaN,NaN,NaN,NaN
75%,NaN,1.000000e+01,NaN,NaN,NaN,NaN


In [26]:
# drop all columns except tconst and nconst
title_principals_df.drop(['ordering','category','job','characters'],axis=1,inplace=True)

In [27]:
title_principals_df.to_pickle(os.path.join(data_path,"title.principals.cleaned.sav"))

# Name Basics Dataset

In [31]:
name_basics_df = pd.read_pickle(os.path.join(master_data_path,"name.basics.sav"))
name_basics_df

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0053137,tt0027125"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack,archive_footage","tt0037382,tt0075213,tt0117057,tt0038355"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0083922,tt0050976,tt0069467"
...,...,...,...,...,...,...
13365072,nm9993714,Romeo del Rosario,NaN,NaN,"animation_department,art_department","tt11657662,tt14069590,tt2455546"
13365073,nm9993716,Essias Loberg,NaN,NaN,NaN,NaN
13365074,nm9993717,Harikrishnan Rajan,NaN,NaN,cinematographer,tt8736744
13365075,nm9993718,Aayush Nair,NaN,NaN,cinematographer,tt8736744


In [32]:
name_basics_df.describe(include='all')

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
count,13365077,13365021,620995.000000,226775.000000,10807637,11898832
unique,13365077,10303104,NaN,NaN,22825,5593396
top,nm0000001,Alex,NaN,NaN,actor,tt0123338
freq,1,479,NaN,NaN,2382021,8338
mean,NaN,NaN,1953.679360,1992.950696,NaN,NaN
std,NaN,NaN,35.547547,36.698038,NaN,NaN
min,NaN,NaN,1.000000,17.000000,NaN,NaN
25%,NaN,NaN,1933.000000,1979.000000,NaN,NaN
50%,NaN,NaN,1960.000000,2001.000000,NaN,NaN
75%,NaN,NaN,1980.000000,2014.000000,NaN,NaN


In [34]:
# name_basics_df.isnull().sum().reset_index().rename(columns={'index':'column', 0:"missing_values"})
# to latex
print(name_basics_df.isnull().sum().reset_index().rename(columns={'index':'column', 0:"missing_values"}).to_latex(index=False))

\begin{tabular}{lr}
\toprule
column & missing_values \\
\midrule
nconst & 0 \\
primaryName & 56 \\
birthYear & 12744082 \\
deathYear & 13138302 \\
primaryProfession & 2557440 \\
knownForTitles & 1466245 \\
\bottomrule
\end{tabular}



In [35]:

name_basics_df.drop(['birthYear','deathYear','primaryProfession'],axis=1,inplace=True)

name_basics_df.dropna(inplace=True)
name_basics_df.describe(include='all')

,nconst,primaryName,knownForTitles
count,11898826,11898826,11898826
unique,11898826,9316736,5593394
top,nm0000001,David Smith,tt0123338
freq,1,378,8338


In [36]:
name_basics_df.to_pickle(os.path.join(data_path,"name.basics.cleaned.sav"))